In [ ]:
"""  
goal: create text fie for tokenizer trainingn 
- data/all.txt 


WHY:
- SentencePiece wants a clean text where each Line is a training sample 
- for tokenizer train we dont need train/test split 
"""

'  \ngoal: create text fie for tokenizer trainingn \n- data/all.txt \n\n\nWHY:\n\n\n\n\n'

In [ ]:
from __future__ import  annotations
import argparse
import hashlib
import re 
from pathlib import Path 
from typing import  Iterable, List 
from tqdm import tqdm 

In [5]:
SAMPLE_TEXT  = "नेपाल एक बहुभाषिक र बहुसांस्कृतिक देश हो। यहाँ हिमाल, पहाड र तराई तीनै भौगोलिक क्षेत्र पाइन्छन्। शिक्षा, स्वास्थ्य र प्रविधिको विकासले देशको भविष्य निर्धारण गर्छ।"

In [6]:
# remove ASCII control chars  (except \n which we handeled reading line by liene)
# \x00-\x08 → NULL, bell, backspace, etc.
# \x0b → vertical tab
# \x0c → form feed
# \x0e-\x1f → more non-printing controls
# \x7f → DEL
# these chars dont showup visually and can break tokenizer 

CONTROL_RE = re.compile((r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]"))
MULTI_SPACE_RE = re.compile(r"[ \t\u00A0]+") 

In [7]:
def clean_line(text):
    # reove zerowith char, rremove control char, strip edges, normalise 

    text = text.replace("\u200b", "") # zero-width space 
    text = text.replace("\ufeff", "") # BOM
    text = CONTROL_RE.sub("",text)
    text = MULTI_SPACE_RE.sub(" ", text)
    return text 


In [8]:
clean_line(SAMPLE_TEXT)

'नेपाल एक बहुभाषिक र बहुसांस्कृतिक देश हो। यहाँ हिमाल, पहाड र तराई तीनै भौगोलिक क्षेत्र पाइन्छन्। शिक्षा, स्वास्थ्य र प्रविधिको विकासले देशको भविष्य निर्धारण गर्छ।'

In [11]:
def build_corpus(input_paths, out_file: Path, min_chars, max_lines):
    out_file.parent.mkdir(parents=True, exist_ok=True)

    total_read = 0
    kept = 0
    with out_file.open("w", encoding="utf-8") as out:
        for path in input_paths:
            if not path.exists():
                raise FileNotFoundError(f"Input file not found: {path}")

            with path.open("r", encoding="utf-8", errors="ignore") as f:
                for raw in tqdm(f, desc=f"Reading {path.name}"):
                    total_read += 1
                    if max_lines is not None and total_read > max_lines:
                        break

                    line = clean_line(raw)
                    if len(line) < min_chars:
                        continue

                    out.write(line + "\n")
                    kept += 1

            if max_lines is not None and total_read > max_lines:
                break

   
    print("Total lines read:", total_read)
    print("Lines kept :", kept)
    print("Output file :", out_file)


In [17]:
build_corpus(
    input_paths= [Path("sample.txt")],
    out_file = Path('data/all.txt'),
    min_chars = 10,
    max_lines=50

)


Reading sample.txt: 3it [00:00, 11859.48it/s]

Total lines read: 3
Lines kept : 3
Output file : data/all.txt
